# Creating a custom kernel

A few weeks ago we discussed using [Markdown in code cells](2018-07-06-Markdown-code-cells.ipynb).  The implementation injected a function into the input transform manager in a running kernel.  

This notebook takes an alternate approach where we create a custom kernel that permits markdown.  *Most of this work was made possible by the demonstration in [creating a simple kernel for Jupyter](https://github.com/ipython-books/cookbook-2nd/blob/master/chapter01_basic/06_kernel.md)*.

## Reusing the our old `Literate` transformer

The Literate transformer will extract the indented code using CommonMark.

In [17]:
    from deathbeds.__Markdown_code_cells import Literate
    transformer = Literate()

## Defining the Kernel Class

The `Kernel` subclasses the existing `IPythonKernel` and strips the indented code before executing the kernel as normal.

In [18]:
    from ipykernel.ipkernel import IPythonKernel
    class LiterateKernel(IPythonKernel):
        implementation = 'Literate'
        implementation_version = '0.0.1'
        banner = "Literate"

        def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
            code = transformer(code)
            return super().do_execute(
                code, 
                silent, store_history=store_history, user_expressions=user_expressions, allow_stdin=allow_stdin)

## Defining the `Kernel` assets

In [19]:
    import json; from pathlib import Path

### The python script

The python script imports this post and creates the module that will run the kernel.

### `kernel.json`

In [28]:
    with __import__('importlib_resources').path('deathbeds', 'literate_kernel.py') as file: 
        kernel_json = Path(file).parent / 'literate' / 'kernel.json'
        kernel_json.parent.mkdir(parents=True, exist_ok=True)
        not kernel_json.exists() and kernel_json.write_text(json.dumps({
             "argv": "python -m deathbeds.literate_kernel -f {connection_file}".split(),
             "display_name": "literate", "name": "literate", "language": "python"}))
        
        
        not Path(file).exists() and Path(file).write_text("""
    with __import__('importnb').Notebook():
        from deathbeds.__Literate_Markdown_Kernel import LiterateKernel

    if __name__ == '__main__':
        from ipykernel.kernelapp import IPKernelApp
        IPKernelApp.launch_instance(kernel_class=LiterateKernel)""");

> Originally we tried running the new kernel with IPython, but that is not permitted.  `python` must invoke the `kernelspec`.  

In short, this notebook defines our literate kernel and `deathbeds.literate_kernel` runs it.

## Installation

Run these commands in the `deathbeds` module directory.

In [29]:
    from IPython import get_ipython; import click
    @click.group()
    def main(): ...

    @main.command()
    @click.option('--env', '-e', help='The environment.')
    def install(env=None):
        prefix = f"source activate {prefix} && " if env else ""
        !$prefix jupyter kernelspec install literate
        !$prefix jupyter kernelspec list

> Nick said that installing with the `--user` flag is less reproducible so we removed it.

In [30]:
    @main.command()
    @click.option('--env', '-e', help='The environment.')
    def uninstall(env=None):
        prefix = f"source activate {prefix} && " if env else ""
        !$prefix jupyter kernelspec uninstall literate
        !$prefix jupyter kernelspec list

In [31]:
    _file_ = '2018-07-20-Literate-Markdown-Kernel.ipynb'

In [32]:
    @main.command()
    def tests(): __import__('pytest').main(['-s', globals().get('__file__', _file_)])

In [ ]:
    if __name__ == '__main__':
        if __import__('sys').argv[0] == globals().get('__file__', ''): 
            main()
        if not globals().get('__file__', None):
            !ipython -m deathbeds.2018-07-20-Literate-Markdown-Kernel -- tests

## Tests

In [ ]:
    def test_import_kernel():
        from deathbeds import literate_kernel
        
    def test_commands():
        # uninstall requires an input otherwise it hangs.
        !ipython -m deathbeds.2018-07-20-Literate-Markdown-Kernel -- install